In [ ]:
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
import cv2
import pygame

# Counter for consecutive frames with closed eyes
COUNTER = 0
# Number of consecutive frames to trigger an alarm
ALARM_EYES_CLOSED = 20
# Number of consecutive frames to reset the alarm
ALARM_EYES_OPEN = 5  # Adjust this value based on how many frames with open eyes should stop the alarm
# Flag to indicate if the alarm is currently on
ALARM_ON = False

# Path to the alarm sound file
ALARM_PATH = 'alarm 0.wav'

# Calculate the eye aspect ratio (EAR)
def eye_aspect(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Initialize pygame mixer
pygame.mixer.init()

def play_alarm_sound():
    pygame.mixer.music.load(ALARM_PATH)
    pygame.mixer.music.play()

def check_for_alarm_stop():
    key = cv2.waitKey(1) & 0xFF
     # Change "s" to any key you want to use to stop the alarm
    pygame.mixer.music.stop()

def detect():
    global COUNTER, ALARM_ON  # Declare COUNTER and ALARM_ON as global variables
    threshold = 0.25

    # Define the face detector and shape predictor
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

    # Define the indices for the left and right eyes in the 68 landmarks
    (lStart, lEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS['left_eye']
    (rStart, rEnd) = face_utils.FACIAL_LANDMARKS_68_IDXS['right_eye']

    # Open the video capture
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = imutils.resize(frame, width=640, height=640)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        subjects = detector(gray, 0)

        for subject in subjects:
            shape = predictor(gray, subject)
            shape = face_utils.shape_to_np(shape)

            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]

            leftEAR = eye_aspect(leftEye)
            rightEAR = eye_aspect(rightEye)

            ear = (leftEAR + rightEAR) / 2.0

            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

            if ear < threshold:
                COUNTER += 1
                if COUNTER >= ALARM_EYES_CLOSED:
                    if not ALARM_ON:
                        ALARM_ON = True
                        play_alarm_sound()  # Play the alarm sound
                        cv2.putText(frame, "Drowsy!", (10, 30),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                if COUNTER >= ALARM_EYES_OPEN:
                    ALARM_ON = False  # Reset the alarm when the eyes have been open for a certain number of frames
                    COUNTER = 0
                    check_for_alarm_stop()
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(30)
        if key == ord("q"):
            break

    cv2.destroyAllWindows()
    cap.release()

if __name__ == '__main__':
    detect()
